# NMT

## 挂载Google Drive，准备数据

In [2]:
from google.colab import drive
drive.mount('/mt')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /mt


In [9]:
! ls /mt/My\ Drive/mt/icytranslate_offline/data

c.clean.ja  c.clean.zh


In [0]:
! mkdir data

In [0]:
! cp /mt/My\ Drive/mt/icytranslate_offline/data/* ./data/

In [14]:
! ls data

c.clean.ja  c.clean.zh


In [17]:
! head data/c.clean.zh data/c.clean.ja

==> data/c.clean.zh <==
《 卡里 加利 博士 的 小屋 》 由 卡尔 · 梅育 以及 汉斯 · 杰 诺维 兹 编写 的 6 场戏 构成
第一场
到处 都 是 幽灵 ， 它们 就 在 我们 周围 它们 驱使 我 远离 健康 和 家庭 远离 妻子 和 孩子
那 是 我 未婚妻
我 与 她 的 经历 比 你 的 经历 离奇 多 了
我 给 你 讲讲 这 段 经历 ...
我 出生 的 那个 小镇 ...
年度 游园 集会
他 ...
我 的 朋友 ， 阿兰 ...

==> data/c.clean.ja <==
カリガリ 博士 の 小屋
幕 １
私 は 亡霊 の おかげ で 家族 や 家庭 を 捨て ざる を え な かっ た
僕 の 婚約 者 だ
僕 と 彼女 の 体験 は あなた の 物語 より も 恐ろし い
今 から それ を 話 し て あげ よう
故郷 ハレシュテンバル で の 出来事 だ
お 祭り が ひら か れ た
ある 香具師 が や っ て き た
親友 の アラン だ


## 清洗数据



In [0]:
import re
def clean_data(input_prefix, output_prefix, src_lang, dest_lang):
  src_input_path = input_prefix + '.' + src_lang
  dest_input_path = input_prefix + '.' + dest_lang
  src_output_path = output_prefix + '.' + src_lang
  dest_output_path = output_prefix + '.' + dest_lang
  long_world_pattern = re.compile(r'[a-zA-z]{5,}')
  with open(src_input_path) as src_rf, open(dest_input_path) as dest_rf:
    with open(src_output_path, 'w') as src_wf, open(dest_output_path, 'w') as dest_wf:
      for src_line, dest_line in zip(src_rf, dest_rf):
        # 去除长度小6的句子和包含5为以上字符的单词
        if len(src_line.replace(' ', '')) < 7 or len(dest_line.replace(' ', '')) < 6 \
        or re.search(long_world_pattern, src_line) or re.search(long_world_pattern, dest_line):
          continue
        else:
          src_wf.write(src_line)
          dest_wf.write(dest_line)


In [0]:
input_prefix = 'data/c.clean'
output_prefix = 'data/corpus'
src_lang = 'ja'
dest_lang = 'zh'
clean_data(input_prefix, output_prefix, src_lang, dest_lang)

In [35]:
! wc -l data/*

  1088446 data/c.clean.ja
  1088446 data/c.clean.zh
   746176 data/corpus.ja
   746176 data/corpus.zh
  3669244 total


In [36]:
! head data/corpus*

==> data/corpus.ja <==
カリガリ 博士 の 小屋
私 は 亡霊 の おかげ で 家族 や 家庭 を 捨て ざる を え な かっ た
僕 の 婚約 者 だ
僕 と 彼女 の 体験 は あなた の 物語 より も 恐ろし い
今 から それ を 話 し て あげ よう
故郷 ハレシュテンバル で の 出来事 だ
お 祭り が ひら か れ た
親友 の アラン だ
" ハレシュテンバル 祭 へ 皆 様 ぜひ お 越し を ！ "
フランシス お 祭り へ い こ う ぜ

==> data/corpus.zh <==
《 卡里 加利 博士 的 小屋 》 由 卡尔 · 梅育 以及 汉斯 · 杰 诺维 兹 编写 的 6 场戏 构成
到处 都 是 幽灵 ， 它们 就 在 我们 周围 它们 驱使 我 远离 健康 和 家庭 远离 妻子 和 孩子
那 是 我 未婚妻
我 与 她 的 经历 比 你 的 经历 离奇 多 了
我 给 你 讲讲 这 段 经历 ...
我 出生 的 那个 小镇 ...
年度 游园 集会
我 的 朋友 ， 阿兰 ...
【 特号 】 霍斯腾 沃尔 游园 集会 各种 从未 出现 过 的 展示
走 吧 , 弗朗西斯 ， 去 集会 玩玩 吧 ！


## 建立词表

In [0]:
import collections
def create_vocab(data_prefix, vocab_prefix, lang, threshold=0):
  vocab = ['<unk>\n', '<s>\n', '<\s>\n']
  data_path = data_prefix + '.' + lang
  vocab_path = vocab_prefix + '.' + lang
  char_pattern = re.compile(r'[b-zB-Z]')
  with open(data_path) as rf, open(vocab_path, 'w') as wf:
    for line in rf: